### OCP Data Preprocessing Tutorial


This notebook provides an overview of converting ASE Atoms objects to PyTorch Geometric Data objects. To better understand the raw data contained within OC20, check out the following tutorial first: https://github.com/Open-Catalyst-Project/ocp/blob/master/docs/source/tutorials/data_playground.ipynb

In [28]:
from ocpmodels.preprocessing import AtomsToGraphs
import ase.io
from ase.build import bulk
from ase.build import fcc100, add_adsorbate, molecule
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.optimize import BFGS
import torch
from torch_geometric.data import Data

### Generate toy dataset: Relaxation of CO on Cu

In [29]:
adslab = fcc100("Cu", size=(2, 2, 3))
ads = molecule("CO")
add_adsorbate(adslab, ads, 3, offset=(1, 1))
cons = FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 3)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(True)
adslab.set_calculator(EMT())
dyn = BFGS(adslab, trajectory="CuCO_adslab.traj", logfile=None)
dyn.run(fmax=0, steps=1000)

False

In [30]:
raw_data = ase.io.read("CuCO_adslab.traj", ":")
print(len(raw_data))

1001


### Convert Atoms object to Data object

The AtomsToGraphs class takes in several arguments to control how Data objects created:

- max_neigh (int):   Maximum number of neighbors a given atom is allowed to have, discarding the furthest
- radius (float):      Cutoff radius to compute nearest neighbors around
- r_energy (bool):    Write energy to Data object
- r_forces (bool):    Write forces to Data object
- r_distances (bool): Write distances between neighbors to Data object
- r_edges (bool):     Write neigbhor edge indices to Data object
- r_fixed (bool):     Write indices of fixed atoms to Data object

In [31]:
a2g = AtomsToGraphs(
    max_neigh=50,
    radius=6,
    r_energy=True,
    r_forces=True,
    r_distances=False,
    r_edges=True,
    r_fixed=True,
)

In [32]:
data_objects = a2g.convert_all(raw_data, disable_tqdm=True)

In [33]:
data = data_objects[0]
data

Data(atomic_numbers=[14], cell=[1, 3, 3], cell_offsets=[636, 3], edge_index=[2, 636], fixed=[14], force=[14, 3], natoms=14, pos=[14, 3], y=3.9893144106684715)

In [34]:
data.atomic_numbers

tensor([29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29.,  8.,  6.])

In [35]:
data.cell

tensor([[[ 5.1053,  0.0000,  0.0000],
         [ 0.0000,  5.1053,  0.0000],
         [ 0.0000,  0.0000, 32.6100]]])

In [36]:
data.edge_index #neighbor idx, source idx

tensor([[ 1,  2,  2,  ...,  4,  6,  3],
        [ 0,  0,  0,  ..., 13, 13, 13]])

In [37]:
data.fixed

tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [38]:
data.force

tensor([[ 2.3250e-15,  1.8807e-15,  1.1354e-01],
        [ 9.0249e-16,  1.1050e-15,  1.1344e-01],
        [ 5.2822e-15,  2.9421e-15,  1.1344e-01],
        [-3.4399e-17,  6.2746e-17,  1.1294e-01],
        [-8.5221e-03, -8.5221e-03, -1.1496e-02],
        [ 8.5221e-03, -8.5221e-03, -1.1496e-02],
        [-8.5221e-03,  8.5221e-03, -1.1496e-02],
        [ 8.5221e-03,  8.5221e-03, -1.1496e-02],
        [ 8.5001e-16, -8.4308e-16, -1.0431e-01],
        [-2.0583e-15, -4.5797e-16, -6.6610e-02],
        [-5.5511e-17, -5.8287e-16, -6.6610e-02],
        [-1.7780e-15, -2.5274e-15, -3.3250e-01],
        [-4.2690e-19, -8.6059e-19, -3.4247e-01],
        [-4.3368e-17, -2.4286e-17,  5.0512e-01]])

In [39]:
data.pos

tensor([[ 0.0000,  0.0000, 13.0000],
        [ 2.5527,  0.0000, 13.0000],
        [ 0.0000,  2.5527, 13.0000],
        [ 2.5527,  2.5527, 13.0000],
        [ 1.2763,  1.2763, 14.8050],
        [ 3.8290,  1.2763, 14.8050],
        [ 1.2763,  3.8290, 14.8050],
        [ 3.8290,  3.8290, 14.8050],
        [ 0.0000,  0.0000, 16.6100],
        [ 2.5527,  0.0000, 16.6100],
        [ 0.0000,  2.5527, 16.6100],
        [ 2.5527,  2.5527, 16.6100],
        [ 2.5527,  2.5527, 19.6100],
        [ 2.5527,  2.5527, 18.4597]])

In [40]:
data.y

3.9893144106684715

### Adding additional info to your Data objects

In addition to the above information, the OCP repo requires several other pieces of information for your data to work
with the provided trainers:

- sid (int): A unique identifier for a particular system. Does not affect your model performance, used for prediction saving 
- fid (int) (S2EF only): If training for the S2EF task, your data must also contain a unique frame identifier for atoms objects coming from the same system.
- tags (tensor): Tag information - 0 for adsorbate, 1 for surface, 2 for subsurface. Optional, can be used for training.


Other information may be added her as well if you choose to incorporate other information in your models/frameworks

In [41]:
data_objects = []
for idx, system in enumerate(raw_data):
    data = a2g.convert(system)
    data.fid = idx
    data.sid = 0 # All data points come from the same system, arbitrarly define this as 0
    data_objects.append(data)

In [42]:
data = data_objects[100]
data

Data(atomic_numbers=[14], cell=[1, 3, 3], cell_offsets=[635, 3], edge_index=[2, 635], fid=100, fixed=[14], force=[14, 3], natoms=14, pos=[14, 3], sid=0, y=3.9683558933957697)

In [43]:
data.sid

0

In [44]:
data.fid

100

In [80]:
# Helper function to convert from PyTorch Geometric input to GROVER input:
def convert_input(data, args):
    """
        :param data: data as PyTorch geometric object
        :param f_atoms: the atom features, num_atoms * atom_dim
        :param f_bonds: the bond features, num_bonds * bond_dim
        :param a2b: mapping from atom index to incoming bond indices. 
        :param a2a: mapping from atom index to its neighbors. num_atoms * max_num_bonds
        :param b2a: mapping from bond index to the index of the atom the bond is coming from.
        :param b2revb: mapping from bond index to the index of the reverse bond.
        :return: batch = (f_atoms, f_bonds, a2b, a2a, b2a, b2revb)
    """
    # Per atom features: (atomic_number, pos_x, pos_y, pos_z)
#     f_atoms = torch.stack((data.atomic_numbers.long(), data.pos[:,0], data.pos[:,1], data.pos[:,2]), 1)
    # Per edge features (calculatedby atomic distances in model forward pass)
#     f_bonds = data.edge_attr
    f_bonds = torch.zeros((100))
    f_atoms = torch.zeros((100))
    args.undirected = False

    # OLD CODE FOR CALCULATING MAPPINGS
    num_atoms_total = int(torch.sum(data.natoms))
    a2a = [[] for j in range(num_atoms_total)]  # List of lists - Dynamically append neighbors for a given atom
    a2b = [[] for j in range(num_atoms_total)]  # List of lists - Dynamically append edges for a given atom
    b2a = torch.zeros((data.edge_index.shape[1],)).long() # (num_edges, ) - One originating atom per edge
    b2revb = torch.zeros((data.edge_index.shape[1],)).long()  # (num_edges, ) - One reverse bond per bond
    rev_edges = {}  # Dict of lists for each (from_atom, to_atom) pair, saving edge numbers

    # Loop through every edge in the graph
    for i in range(data.edge_index.shape[1]):
        from_atom = int(data.edge_index[0][i])
        to_atom = int(data.edge_index[1][i])
        a2a[from_atom].append(to_atom)  # Mark b as neighbor of a
        a2b[to_atom].append(i)  # Mark bond i as incoming bond to atom a
        b2a[i] = from_atom  # Mark a as atom where bond i is originating
        key = frozenset({to_atom, from_atom})
        if (key not in rev_edges):  # If the edge from these two atoms has not been seen yet
            rev_edges[key] = []  # Declare it as a list (so we can keep track of the edge numbers)
        rev_edges[key].append(i)  # Append the edge number to the list

    # Iterate through and set b2revb with reverse bonds
    for atoms, edges in rev_edges.items():
        if(len(edges) == 2):
            b2revb[edges[0]] = edges[1]
            b2revb[edges[1]] = edges[0]
        elif(len(edges) == 4): # In the case of duplicate edges, they are grouped together in this order
            b2revb[edges[0]] = edges[2]
            b2revb[edges[2]] = edges[0]
            b2revb[edges[1]] = edges[3]
            b2revb[edges[3]] = edges[1]
        elif(len(edges) == 1):
            args.undirected = False

    # Convert list of lists for a2a and a2b into tensor: (num_nodes, max_edges)
    # Trim length to max number of edges seen in the data (should be capped by 50 but not always in practice)
    a2a_pad = len(max(a2a, key=len))
    a2b_pad = len(max(a2b, key=len))
    # -1 is not a valid atom or edge index so we pad with this
    a2a = torch.tensor([i + [0] * (a2a_pad - len(i)) for i in a2a])
    a2b = torch.tensor([i + [0] * (a2b_pad - len(i)) for i in a2b])

    batch = (f_atoms, f_bonds, a2b, a2a, b2a, b2revb)
    return batch

In [165]:
from argparse import Namespace
path = "/home/etower/ocp/logs/data_dump/80hidden"
a2a2 = torch.load(path + "/a2a2.pt").cpu()
a2b2 = torch.load(path + "/a2b2.pt").cpu()
b2a2 = torch.load(path + "/b2a2.pt").cpu()
data = torch.load(path + "/data.pt")
print("a2a2 len: ", len(a2a2))

args = Namespace()
print("data.edge_index: ", data.edge_index)
print("data.natoms: ", data.natoms)
print("data.batch: ", data.batch)
print("len data batch: ", len(data.batch))
print("natoms total: ", torch.sum(data.natoms))
print("natoms entries that are equal to zero: ", len(data.natoms[data.natoms==0]))

batch = convert_input(data, args)
f_atoms, f_bonds, a2b, a2a, b2a, b2revb = batch
a2a = a2a.cpu()
a2b = a2b.cpu()
b2a = b2a.cpu()
data.edge_index = data.edge_index.cpu()


# edge_index[0] = data.edge_index[0].split(tuple(data.batch))
# edge_index[1] = data.edge_index[1].split(tuple(data.batch))

a2a2 len:  7020
data.edge_index:  tensor([[   3,    4,    7,  ..., 7016, 7017, 7018],
        [   0,    0,    0,  ..., 7019, 7019, 7019]], device='cuda:0')
data.natoms:  tensor([118,  77,  47,  54, 100,  78,  80,  33,  87,  57,  84, 132,  85,  29,
         61,  55, 132, 102, 103,  61,  33,  40,  70,  87,  78, 122,  41,  45,
         83, 136,  68,  77,  36,  98,  62,  74, 118,  80,  54,  70,  30,  78,
         76,  71,  36,  27,  46,  60,  79,  57,  88,  86, 100,  53,  56,  38,
        106,  12,  80,  71, 149,  41,  87,  70,  64,  52,  97,  54, 102,  56,
         52, 111,  37,  39,  65,  36,  44, 136,  74,  77, 103,  99,  40,  63,
        104, 102,  28,  63,  50,  51,  38,  67,  52,  73,  70,  44,  76,  78,
         39,  41], device='cuda:0')
data.batch:  tensor([ 0,  0,  0,  ..., 99, 99, 99], device='cuda:0')
len data batch:  7021
natoms total:  tensor(7021, device='cuda:0')
natoms entries that are equal to zero:  0


In [178]:
# NEW WAY OF CALCULATING a2a2, a2b2, b2a2: Add padding to match num_atoms_total length
# Skip to use a2a2 from ocp run/log dump

num_atoms_total = int(torch.sum(data.natoms))

# Calculate atom to neighboring atom mappings (a2a)
trans = data.edge_index.T
sorted_index = trans[trans[:, 0].sort()[1]]  # Sort by column zero (from_node), index based off
sorted_index = sorted_index.T

out, counts = torch.unique(sorted_index[0], return_counts=True)  # counts for each atom
if len(out) - 1 != out[-1]: # If they are the correct length then we don't need to add 0's
    ascending = torch.arange(out[-1] + 1).cuda() # Ascending order list [0, 1, ... n] where n is last elem in out
    missing = [ elem not in out for elem in ascending ] # Missing elements in out
    for elem in numpy.nonzero(missing)[0]:
        i = int(elem)
        counts = torch.cat((counts[:i], torch.tensor((0,)).cuda(), counts[i:])) # Add zero to count
a2a2 = sorted_index[1].split(counts.tolist())  # Index into to_bonds with these indices
a2a2 = torch.nn.utils.rnn.pad_sequence(list(a2a2), batch_first=True, padding_value=0) # Pad with zeros into a tensor
# New line

if len(a2a2 < num_atoms_total):
    a2a2 = torch.cat((a2a2, torch.zeros((1, a2a2.shape[1])))) # Add extra entry of zeros to match dimensions

# Calculate outgoing bond to atom mappings (b2a)
b2a2 = data.edge_index[0].type(torch.LongTensor)

# Calculate atom to incoming bond mappings (a2b)
out, counts = torch.unique(data.edge_index[1], return_counts=True)
if len(out) - 1 != out[-1]: # If they are the correct length then we don't need to add 0's
    ascending = torch.arange(out[-1] + 1).cuda() # Ascending order list [0, 1, ... n] where n is last elem in out
    missing = [ elem not in out for elem in ascending ] # Missing elements in out
    for elem in numpy.nonzero(missing)[0]:
        i = int(elem)
        counts = torch.cat((counts[:i], torch.tensor((0,)).cuda(), counts[i:])) # Add zero to count

a2b2 = torch.split(torch.arange(len(data.edge_index[1])), tuple(counts))
a2b2 = torch.nn.utils.rnn.pad_sequence(a2b2, batch_first=True, padding_value=0) # Pad with zeros into a tensor

# New line
if len(a2b2 < num_atoms_total): 
    a2b2 = torch.cat((a2b2, torch.zeros((1, a2b2.shape[1])))) # Add extra entry of zeros to match dimensions

a2a2 = a2a2.cpu().type(torch.LongTensor)
a2b2 = a2b2.cpu().type(torch.LongTensor)
b2a2 = b2a2.cpu()

In [179]:
print("a2a2 shape: ", a2a2.shape)
print("a2a shape:  ", a2a.shape)
print("a2b2 shape: ", a2b2.shape)
print("a2b shape:  ", a2b.shape)
print("b2a2 shape: ", b2a2.shape)
print("b2a shape:  ", b2a.shape)
print("edge_index shape: ", edge_index.shape)

out0, counts0 = torch.unique(edge_index[0], return_counts=True) # counts for each atom
print("\nlen unique in edge index[0]: ", len(out0))

out1, counts1 = torch.unique(edge_index[1], return_counts=True) # counts for each atom
print("len unique in edge index[1]: ", len(out1))

print("out0: ", out0)
print("out1: ", out1)

out_total =  np.union1d(out0.cpu(), out1.cpu())
print("len unique total: ", len(out_total))

# Count unique atoms manually with for loop
out0_manual = {}
out1_manual = {}

missing_a2a = {}
missing_a2a2 = {}
# Check that a2a is correct: that for every atom, it is actually connected with the corresponding one
for i in range(len(data.edge_index[0])):
    from_atom = int(data.edge_index[0][i].cpu())
    to_atom = int(data.edge_index[1][i].cpu())
    if to_atom not in a2a[from_atom]:
        if from_atom not in missing_a2a:
            missing_a2a[from_atom] = []
        missing_a2a[from_atom].append(to_atom)
    if to_atom not in a2a2[from_atom]:
        if from_atom not in missing_a2a2:
            missing_a2a2[from_atom] = []
        missing_a2a2[from_atom].append(to_atom)
    if from_atom not in out0_manual:
        out0_manual[from_atom] = 1
    if to_atom not in out1_manual:
        out1_manual[to_atom] = 1

a2a2 shape:  torch.Size([7021, 66])
a2a shape:   torch.Size([7021, 66])
a2b2 shape:  torch.Size([7021, 33])
a2b shape:   torch.Size([7021, 33])
b2a2 shape:  torch.Size([146009])
b2a shape:   torch.Size([146009])
edge_index shape:  torch.Size([2, 146009])

len unique in edge index[0]:  7020
len unique in edge index[1]:  7020
out0:  tensor([   0,    1,    2,  ..., 7017, 7018, 7019], device='cuda:0')
out1:  tensor([   0,    1,    2,  ..., 7017, 7018, 7019], device='cuda:0')
len unique total:  7020


In [180]:
# Calculate number of unique elements in edge index manually
out0_m = list(out0_manual)
out0_m = [ int(i) for i in out0_m ]

out1_m = list(out1_manual)
out1_m = [ int(i) for i in out1_m ]

sort_0 = np.sort(out0_m)
print(len(sort_0))
print(len(out1_m))
print(len(np.union1d(sort_0, out1_m)))

7020
7020
7020


In [181]:
print("missing a2a: ", missing_a2a)
print("missing a2a2: ", missing_a2a2)

print("\nb2a and b2a2 equal? ", torch.equal(b2a, b2a2))
print("b2a shape: ", b2a.shape)
print("b2a2 shape: ", b2a2.shape)

print("\na2b and a2b2 equal? ", torch.equal(a2b, a2b2))
print("a2b shape: ", a2b.shape)
print("a2b2 shape: ", a2b2.shape)

print("\na2a and a2a2 equal? ", torch.equal(a2a, a2a2))
print("a2a and a2a2 equal counts: ",
      torch.equal(torch.unique(a2a, return_counts=True)[1], torch.unique(a2a2, return_counts=True)[1]))
print("a2a count: ", torch.unique(a2a, return_counts=True)[1])
print("a2a2 count: ", torch.unique(a2a2, return_counts=True)[1])
print("a2a shape: ", a2a.shape)
print("a2a2 shape: ", a2a2.shape)

missing a2a:  {}
missing a2a2:  {}

b2a and b2a2 equal?  True
b2a shape:  torch.Size([146009])
b2a2 shape:  torch.Size([146009])

a2b and a2b2 equal?  True
a2b shape:  torch.Size([7021, 33])
a2b2 shape:  torch.Size([7021, 33])

a2a and a2a2 equal?  False
a2a and a2a2 equal counts:  True
a2a count:  tensor([317396,      9,      8,  ...,     16,     13,     17])
a2a2 count:  tensor([317396,      9,      8,  ...,     16,     13,     17])
a2a shape:  torch.Size([7021, 66])
a2a2 shape:  torch.Size([7021, 66])


In [114]:
print("a2a[-1]: ", a2a[-1])
print("\na2a[-2]:  ", a2a[-2])
print("a2a2[-1]: ",a2a2[-1])

print("\na2a[-3]:  ", a2a[-3])
print("a2a2[-2]: ", a2a2[-2])

a2a[-1]:  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

a2a[-2]:   tensor([6980, 6981, 6986, 6987, 6992, 6993, 6994, 6995, 7004, 7005, 7010, 7012,
        7016, 7017, 7018,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0])
a2a2[-1]:  tensor([6980, 6981, 6986, 6987, 6992, 6993, 6994, 6995, 7004, 7005, 7010, 7012,
        7016, 7017, 7018,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    

In [113]:
print("a2b[-1]: ", a2b[-1])
print("\na2b[-2]:  ", a2b[-2])
print("a2b2[-1]:  ",a2b2[-1])

print("\na2b[-3]:  ", a2b[-3])
print("a2b2[-2]: ", a2b2[-2])

a2b[-1]:  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])

a2b[-2] tensor([145992, 145993, 145994, 145995, 145996, 145997, 145998, 145999, 146000,
        146001, 146002, 146003, 146004, 146005, 146006, 146007, 146008,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0])
a2b2[-1]:  tensor([145992, 145993, 145994, 145995, 145996, 145997, 145998, 145999, 146000,
        146001, 146002, 146003, 146004, 146005, 146006, 146007, 146008,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0])

a2b[-3] tensor([145979, 145980, 145981, 145982, 145983, 145984, 145985, 145986, 145987,
        145988, 145989, 145990, 145991,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
           

Resources:

- https://github.com/Open-Catalyst-Project/ocp/blob/6604e7130ea41fabff93c229af2486433093e3b4/ocpmodels/preprocessing/atoms_to_graphs.py
- https://github.com/Open-Catalyst-Project/ocp/blob/master/scripts/preprocess_ef.py